# Using LaVague for QA Automation

In this notebook, we'll look at how LaVague can be used to generate automated tests from simple test scenario.

We will use LaVague to autonomously run the test and record xpath and actions. We'll then use an LLM to generate assert statements and the final reusable test file. 

We first define our test with the following variables

In [ ]:
URL = "https://www.amazon.com/" # the URL you want to run your test on

FEATURE_FILE_NAME = "amazon_cart.feature" # name of the feature file that will be generated
PYTEST_FILE_NAME = "amazon_cart_test.py" # name of the pytest file that will be generated

# the high level test case that will be used to generate the feature file and the test file
HIGH_LEVEL_TEST_CASE = """
Search for `1984 75th anniversary edition` and press Enter. 
Add it to cart and verify that the cart feature works
"""

OBJECTIVE = f"Perform the following test:\n {HIGH_LEVEL_TEST_CASE}" # the objective we give LaVague

# We could make this framework agnostic by rewriting the prompt and examples passed to the multi-modal LLM
# TARGET_FRAMEWORK = "pytest-bdd"

## Init LaVague

In [ ]:
import base64
from io import BytesIO

from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver

from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.legacy.readers.file.base import SimpleDirectoryReader

In [ ]:
# init a standard LaVague agent
selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

Load the URL and manually pass CAPTCHA before running the agent (if required)

In [ ]:
agent.get("https://www.amazon.com/")

## Run the agent
Run the high level test case once. Get logs as a dataframe. 

In [ ]:
agent.run(OBJECTIVE, log_to_db=True)

logs = agent.logger.return_pandas()

## Extract navigation data from the agent run
### Get relevant nodes to generate the assert statement

In [ ]:
nodes = action_engine.navigation_engine.get_nodes(
    f"We have ran the test case, generate the final assert statement.\n\ntest case:\n{HIGH_LEVEL_TEST_CASE}"
)

nodes

### Get all actions performed and screenshot of last page state

From logs, clean the "code" column to remove CoT comments then create a new dataframe that contains each instruction with the associated xpath and action taken

In [ ]:
# get all actions and cleaup
actions = "\n".join(logs["code"].dropna())

def remove_comments(code):
    return '\n'.join([line for line in code.split('\n') if not line.strip().startswith('#')])

logs['action'] = logs['code'].dropna().apply(remove_comments)
cleaned_logs = logs[['instruction', 'action']].fillna('')
actions = '\n\n'.join(cleaned_logs['instruction'] + ' ' + cleaned_logs['action'])

instructions = "\n".join(cleaned_logs['instruction'])
print(actions)

# get last page screenshot
last_page_screenshot = SimpleDirectoryReader(logs.iloc[-1]["screenshots_path"]).load_data() # load last screenshot taken


# Output generation
Use all gathered data about the site to generate a Gherkin, then a pytest-bdd file

We'll first create two LLMs
- multi modal LLM `gpt-4o` is used to generate the final code file (we pass a screenshot of the last page state to help with the assert generation)
- text only LLM `gpt-4` to generate the feature file

In [ ]:
gpt4o = OpenAIMultiModal("gpt-4o")
gpt4o.max_new_tokens = 2000 # 300 by default, we increase it to make sure our pytest file doesn't get trucated

gpt4 = OpenAI("gpt-4")

# we'll clean the triple quote answer from the LLMs
def clean_output(markdown_code_block):
    return markdown_code_block.replace("```python", "").replace("```", "").replace("```\n", "")

### Generate Gherkin

In [ ]:
PROMPT_GHERKIN = f"""
Generate a Gherkin-style test case for the following initial objective and actions already ran by the agent. 
Only output in a markdown triple quoted string and nothing else.
Initial objective = {OBJECTIVE}\n\nInstructions ran: {instructions}
"""

In [ ]:
generated_gherkin = gpt4.complete(PROMPT_GHERKIN).text
generated_gherkin = clean_output(generated_gherkin)
print(generated_gherkin)

### Generate pytest file

In [ ]:
SYSTEM_PROMPT = f"""
You are an expert in software testing frameworks and Python code generation. You answer in python markdown only and nothing else, don't include anything after the last backticks. Your task is to:
1. Process a test case, existing actions (with their associated xpath) that was ran during the test by an agent, relevant HTML nodes, and the screenshot of the last state of the page.
2. Generate an assert statement for the last condition of the test case. THis is the most important, you need to generate asserts at all costs
3. Package everything in a pytest-bdd file following best practices. 
Requirements:
- Use descriptive function names and name the scenario appropriately.
- Use execute_script to handle potential ElementClickInterceptedException during clicks.
- Include fixtures, scenario, Gherkin-style definitions, etc.
- Use try-except blocks to catch exceptions and raise pytest.fail for assert condition steps as needed.
- Do not generate asserts for 'Given', 'When', or 'And' steps; only generate asserts for 'Then' steps.
- Always use provided selenium code that was already executed to find valid selectors for the final pytestfile. 
- You answer in python code only and nothing else.
"""

EXAMPLES = """
import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pytest_bdd import scenarios, given, when, then
# Constants
BASE_URL = 'https://form.jotform.com/241472287797370'
# Scenarios
scenarios('test_form_submission.feature')
# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(BASE_URL)
    yield driver
    driver.quit()
# Steps
@given('I am on the job application page')
def i_am_on_the_job_application_page(browser):
    pass
@when('I enter "John" in the "First Name" field')
def i_enter_first_name(browser):
    first_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[1]/input")
    first_name_field.send_keys("John")
@when('I enter "Doe" in the "Last Name" field')
def i_enter_last_name(browser):
    last_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[2]/input")
    last_name_field.send_keys("Doe")
@when('I enter "john.doe@example.com" in the "Email Address" field')
def i_enter_email_address(browser):
    email_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[3]/div/span/input")
    email_field.send_keys("john.doe@example.com")
@when('I enter "(123) 456-7890" in the "Phone Number" field')
def i_enter_phone_number(browser):
    phone_number_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[4]/div/span/input")
    phone_number_field.send_keys("(123) 456-7890")
@when('I leave the "Cover Letter" field empty')
def i_leave_cover_letter_empty():
    # No action needed as the field should remain empty
    pass
@when('I click the "Apply" button')
def i_click_apply_button(browser):
    apply_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[1]/ul/li[6]/div/div/button"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", apply_button)
    apply_button.click()
@then('I should see an error message for the "Cover Letter" field')
def i_should_see_error_message(browser):
    try:
        error_message = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[5]/div/div/span")
        assert error_message.is_displayed()
    except Exception as e:
        pytest.fail(f"Error message not displayed: {e}")
"""



PROMPT = f"""{SYSTEM_PROMPT}

Generate a valid pytest-bdd file with the following inputs and examples to guide you:
Feature file name: {FEATURE_FILE_NAME}
Gherkin feature: {generated_gherkin}\n
Already executed code:\n{actions}\n
selected html of the last page: {nodes}\n
Examples:\n\n{EXAMPLES}
"""

### Generate the automated test

In [ ]:
generated_pytest = gpt4o.complete(PROMPT, image_documents=last_page_screenshot).text
generated_pytest = clean_output(generated_pytest)
print(generated_pytest)

### Cleanup and write test to file

We'll cleanup and write two generated files to disk: 
- `.feature` contains the test scenarios written in the Gherkin syntax
- `.py` the actual automated test that we'll run with pytest

In [ ]:
import os
with open(FEATURE_FILE_NAME, "w") as file:
        file.write(generated_gherkin)
        
with open(PYTEST_FILE_NAME, "w") as file:
        print("WRITING FILE")
        file.write(generated_pytest)

## Run tests


In [ ]:
!pytest -v {PYTEST_FILE_NAME}